# Transfer Learning
Use this notebook to run transfer learning on an individual subject. This assumes you have run `Create_General_Models.ipynb` already.

Nile Wilson, 2019.03.30

In [ ]:
#imports

## Function Definitions
Define some functions here

## Load Models
Load in your models

### Select appropriate templates for error classifier
Either create average template or match whichever template works best for this current subject of interest. **NOTE:** you will need to screen for what channels this subject has and limit the templates to the ones that have the same montage.

## Simulate Performance